# Dogs Vs. Cats: Training a Covnet Using a Pretrained Model 

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np 
import matplotlib.pyplot as plt 

In [3]:
# Import the VGG16 Model
conv_base = keras.applications.vgg16.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(180, 180, 3))

58892288/58889256 [==============================] - 3s 0us/step


In [4]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 180, 180, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 180, 180, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 180, 180, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 90, 90, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 90, 90, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 90, 90, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 45, 45, 128)       0     

In [5]:
# Freeze the VGG16 model by setting the trainable attribute to False.
conv_base  = keras.applications.vgg16.VGG16(
    weights='imagenet',
    include_top=False)
conv_base.trainable = False

In [7]:
# Create data augmentation layers
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomZoom(0.2),
    ]
)

0

In [ ]:
# Build the model
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)

x = conv_base(x)

x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
# Define the callbacks and save the best model to a new file
callbacks = [keras.callbacks.ModelCheckpoint(
    filepath='feature_extraction_with_data_augmentation.keras',
    save_best_only=True,
    monitor='val_loss')]

In [ ]:
import time
# Time the time it takes to train the model
start_time = time.time()

history = model.fit(
  train_dataset,
  epochs=50,
  validation_data=validation_dataset,
  callbacks=callbacks)

print("--- Model trained in %s seconds ---" % (time.time() - start_time))

In [ ]:
# Plot the loss and accuracy of the model over the training and validation data during training. 
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
# Display the accuracy of the model on the test dataset
test_model = keras.models.load_model('feature_extraction_with_data_augmentation.keras')
test_loss, test_acc = test_model.evaluate(test_dataset)
print('Test accuracy: %.3f' % (test_acc,))

## Fine Tuning a Pretrained Model

In [ ]:
# Unfreeze the top four layers
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
  layer.trainable = False

In [ ]:
# Recompile the model (since we changed the trainable layers)
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
              metrics=['accuracy'])


In [ ]:
# Define callbacks again, with a new save file
callbacks = [keras.callbacks.ModelCheckpoint(
    filepath='fine_tuning.keras',
    save_best_only=True,
    monitor='val_loss')]


In [ ]:
start_time = time.time()

history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

print("--- Model trained in %s seconds ---" % (time.time() - start_time))

In [ ]:
# Display the accuracy of the model on the test dataset
model = keras.models.load_model('fine_tuning.keras')
test_loss, test_acc = model.evaluate(test_dataset)
print('Test accuracy: %.3f' % (test_acc,))